In [2]:
!pip install layoutparser[ocr]

In [3]:
!pip install 'https://github.com/facebookresearch/detectron2.git'

     - 285.7 kB 11.4 MB/s 0:00:00
  ERROR: Cannot unpack file /tmp/pip-unpack-t0_qe_mm/detectron2.git (downloaded from /tmp/pip-req-build-br1qf4ou, content-type: text/html; charset=utf-8); cannot detect archive format
ERROR: Cannot determine archive format of /tmp/pip-req-build-br1qf4ou


In [ ]:
! pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2'


  Cloning https://github.com/facebookresearch/detectron2.git (to revision v0.6) to /tmp/pip-install-j9996g8l/detectron2_f7f5af0ad3d8450a9d290452e8601045
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-install-j9996g8l/detectron2_f7f5af0ad3d8450a9d290452e8601045
  Running command git checkout -q d1e04565d3bec8719335b88be9e9b961bf3ec464
  Resolved https://github.com/facebookresearch/detectron2.git to commit d1e04565d3bec8719335b88be9e9b961bf3ec464
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.5 MB/s eta 0:0

In [ ]:
!sudo apt install tesseract-ocr

In [ ]:
!pip install google-cloud-vision==0.29.0

In [ ]:
!pip install "paddleocr>=2.0.1"

In [ ]:
!pip install Pillow==9.5

In [ ]:
! wget https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
!pip install -U layoutparser-0.0.0-py3-none-any.whl

In [ ]:
!pip install paddlepaddle

In [ ]:
!pip install layoutparser[ocr]

In [ ]:
! pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2'

In [ ]:
!pip install google-cloud-vision==0.29.0


In [ ]:
# %cd /content/
# # install detectron2:
# !git clone https://github.com/facebookresearch/detectron2
# %cd /content/detectron2
# !pip install -r requirements.txt
# !python setup.py install
# !pip install git+https://github.com/facebookresearch/fvcore.git

In [ ]:
import numpy as np
import layoutparser as lp
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
image = cv2.imread("/content/PM News December 2_1994_Pg 1.jpg")
image = image[..., ::-1]
    # Convert the image from BGR (cv2 default loading style)
    # to RGB

In [ ]:
cv2_imshow(image)

In [ ]:
image.shape

In [ ]:
# Use the cvtColor() function to grayscale the image
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

cv2_imshow(gray_image)

In [ ]:
gray_image.shape

In [ ]:
# Expand the single-channel grayscale to three channels
gray_image_three_channels = np.expand_dims(gray_image, axis=2)
gray_image_three_channels = np.repeat(gray_image_three_channels, 3, axis=2)

In [ ]:
cv2_imshow(gray_image_three_channels)

In [ ]:
model = lp.models.Detectron2LayoutModel('lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config',
                                 extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
                                 label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"})
    # Load the deep layout model from the layoutparser API
    # For all the supported model, please check the Model
    # Zoo Page: https://layout-parser.readthedocs.io/en/latest/notes/modelzoo.html

In [ ]:
# # load model
# model = lp.models.PaddleDetectionLayoutModel(config_path="lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config",
#                                 threshold=0.5,
#                                 label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"},
#                                 enforce_cpu=False,
#                                 enable_mkldnn=True
#                                              )

In [ ]:
layout = model.detect(image)
    # Detect the layout of the input image

In [ ]:
lp.draw_box(image, layout, box_width=3)
    # Show the detected layout of the input image

In [ ]:
type(layout)

In [ ]:

layout[0]

In [ ]:
text_blocks = lp.Layout([b for b in layout if b.type=='Text'])
figure_blocks = lp.Layout([b for b in layout if b.type=='Figure'])

In [ ]:
for txt in text_blocks.get_texts():
    print(txt, end='\n---\n')

In [ ]:

# text_blocks = lp.Layout([b for b in text_blocks \
#                    if not any(b.is_in(b_fig) for b_fig in figure_blocks)])

In [ ]:
ocr_agent = lp.TesseractAgent(languages='eng')
    # Initialize the tesseract ocr engine. You might need
    # to install the OCR components in layoutparser:
    # pip install layoutparser[ocr]

In [ ]:
for block in text_blocks:
    segment_image = (block
                       .pad(left=5, right=5, top=5, bottom=5)
                       .crop_image(image))
        # add padding in each image segment can help
        # improve robustness

    text = ocr_agent.detect(segment_image)
    block.set(text=text, inplace=True)

In [ ]:
for txt in text_blocks.get_texts():
    print(txt, end='\n---\n')